In [ ]:
#!pip install numpy pandas seaborn matplotlib torch pydicom 

In [ ]:
!pip install seaborn matplotlib torch pydicom numpy pandas


In [1]:
import matplotlib.pyplot as plt 

Matplotlib is building the font cache; this may take a moment.


KeyError: '_items'

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
import numpy as np
import glob
import json
import collections
import torch
import torch.nn as nn

import pydicom as dicom
import matplotlib.patches as patches

from matplotlib import animation, rc
import pandas as pd

import pydicom as dicom # dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

Matplotlib is building the font cache; this may take a moment.


KeyError: '_items'

In [ ]:
# read data
train_path = '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/RSNA'

train  = pd.read_csv(train_path + 'train.csv')
label = pd.read_csv(train_path + 'train_label_coordinates.csv')
train_desc  = pd.read_csv(train_path + 'train_series_descriptions.csv')
test_desc   = pd.read_csv(train_path + 'test_series_descriptions.csv')
sub         = pd.read_csv(train_path + 'sample_submission.csv')